## Imports

In [ ]:
from vpython import *
import numpy as np
import matplotlib.pyplot as pyplt
import random

## Simulation

In [ ]:
# setup attack variables
injection_attack = False
deny_attack = False
delay_attack = False

In [ ]:
# scene setup
scene = canvas(title="Inverted Pendulum Balancer", width=800, height=400)

#----------------------------------------------------------------------
# define a cylinder to act as our pendulum
pend = cylinder(pos=vec(0, 0, 0), axis=vec(0, 1, 0), radius=0.04, length=1, color=vec(1, 0, 0))
pend.mass = 1
pend.theta = np.pi / 2 - 0.1
pend.thetaDot = 0

# define a clyinder to act as the rail our pendulum is attached to
rail = cylinder(pos=vec(0, 0, 0), axis=vec(1, 0, 0), radius=0.1, length=30, color=vec(0, 1, 0))
rail.pos.x = -rail.length / 2
rail.pos.y = -rail.radius

positionOutput = wtext(text='Position={:f}'.format(0))

#----------------------------------------------------------------------
class PerceivedPendulum:
    # This class holds the system's perceived values for the pendulum.
    # What it "believes" the pendulum's pos, angle, angular velocity are.
    # These values can be altered by attacks.
    def __init__(self, pend):
        self.pos = vec(pend.pos.x, 0, 0)
        self.theta = pend.theta
        self.thetaDot = pend.thetaDot

perceived = PerceivedPendulum(pend)

perceived.pos.x = pend.pos.x
perceived.theta = pend.theta
perceived.thetaDot = pend.thetaDot

#----------------------------------------------------------------------
def endSim():
    global targetPos
    targetPos *= -1

endButton = button(text="End Sim", bind=endSim)

#----------------------------------------------------------------------
# constants
G = 9.8
damping = 0.02
coefficientOfRestitution = 0.8
collisions = False

targetPos = 5

# time
t = 0
endTime = 30
dt = 0.01
speedMultiplier = 1

# data storage
time_data = []
position_data = []
perceived_pos_data = []
angle_data = []
perceived_angle_data = []

#----------------------------------------------------------------------
def getControlSignal(perceived, xPos):
    # simple PD controller
    theta_error = np.pi / 2 - perceived.theta
    theta_dot_error = -perceived.thetaDot
    position_error = perceived.pos.x - xPos

    # gain values
    Kp_theta = 10
    Kd_theta = 4
    Kp_position = 2

    return Kp_theta * theta_error + Kd_theta * theta_dot_error + Kp_position * position_error

#----------------------------------------------------------------------
def updatePend(pend, dt):

    # if injection attack, inject false data
    if injection_attack:
        inject_false_data(perceived)

    xDot = getControlSignal(perceived, targetPos) # use percieved values, as this is what the system believes its values are

    # apply control signal to perceived and real pendulum values
    pend.pos.x += xDot*dt    
    pend.theta += np.arctan(xDot*dt/pend.length) # my system hates atan() and needs arctan() spelled out
    pend.theta %= np.pi*2

    # only update perceived if not a denial attack
    if not denial_attack:
        perceived.pos.x += xDot*dt
        perceived.theta += np.arctan(xDot*dt/pend.length)
        perceived.theta %= np.pi*2
    
    torque = -G*pend.mass*pend.length/2*np.cos(perceived.theta) - perceived.thetaDot*damping
    moment = 0.333*pend.mass*pend.length**2
    thetaDoubleDot = torque/moment
    
    pend.thetaDot += thetaDoubleDot*dt
    pend.theta += pend.thetaDot*dt
    
    if not denial_attack:
        perceived.thetaDot += thetaDoubleDot*dt
        perceived.theta += pend.thetaDot*dt

    # update visuals
    pend.axis = rotate(vec(1, 0, 0), pend.theta, vec(0, 0, 1))*pend.length
    positionOutput.text = 'Position={:f}'.format(pend.pos.x)
    
    return pend

#----------------------------------------------------------------------
# main loop
while t < endTime:
    pend = updatePend(pend, dt * speedMultiplier)

    # collect real data
    time_data.append(t)
    position_data.append(pend.pos.x)
    degrees = pend.theta * (180/np.pi) # convert angle to degrees
    angle_data.append(degrees)

    # collect perceived data
    perceived_pos_data.append(perceived.pos.x)
    perceived_deg = perceived.theta * (180/np.pi) # convert angle to degrees
    perceived_angle_data.append(perceived_deg)

    t += dt
    rate(1 / dt)

    # scene display
    display = (
    f"Simulation time: {t:.2f}/{endTime:.2f}\n"
    f"Real Position: {pend.pos.x:.2f}\n"
    f"Perceived Pos: {perceived.pos.x:.2f}\n"
    f"Real Angle: {degrees:.2f}\n"
    f"Perceived Angle: {perceived_deg:.2f}\n"
    )
    scene.caption = (display)

#----------------------------------------------------------------------
# plotting results
pyplt.plot(time_data, angle_data, label='Real Angle', color='blue')
pyplt.plot(time_data, perceived_angle_data, label='Perceived Angle', color='red', linestyle='--')

pyplt.ylabel("Angle")
pyplt.xlabel("Time (s)")
pyplt.legend()
pyplt.grid()
pyplt.show()

#### Injection Attack
Run the following block **prior to simulation** to emulate a False Data Injection attack.

**targeted_data** can be: 
"position" to trick the system into thinking it is in the wrong location, or
"theta" to trick the system into thinking the pendulum is tipping

In [ ]:
injection_attack = True
injection_rate = 1
attack_severity = 0.1
targeted_data = "theta"

def inject_false_data(perceived):
    # shift percieved vars from actual values
    if random.random() < injection_rate:
        shift = random.uniform(-attack_severity, attack_severity)

        if targeted_data == "position":
            perceived.pos.x += shift
        elif targeted_data == "theta":
            perceived.theta += shift

#### Deny Attack
Run the following block **prior to simulation** to emulate a Data Denial attack.

In [ ]:
denial_attack = True

#### Delay Attack
Run the following block **prior to simulation** to emulate a Data Delay attack.

In [ ]:
delay_attack = True